# Scrape the DOH daily totals page
Cut and paste into the totals.csv when it looks okay.

Probably could import totals.csv and then update if the new date is later than the old. Possibly with a loop until that happens. But, still have to read the footnotes to get new cases. So, doesn't seem worth it for now. 

Longer term the CSVDownload.csv on the dashboard might be a better way to get daily cases. <s>However, that is only available via the Tableau button on the charts right now.</s>
    
See tests_to_freq in the doh_data folder for automatic downloading of the CSV data.    

The problem with that data is that it doesn't include "Outside Hawaii". Maybe that's better since the official benchmarks don't either?

In [1]:
import requests
import urllib.request
import time           #in case we want to hit the server in a loop and need a delay
from bs4 import BeautifulSoup
import re
from dateutil.parser import parse

In [2]:
url = 'https://health.hawaii.gov/coronavirusdisease2019/'
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
for t in soup('dd'):
    print(t.get_text())

Total cases: 14233* (78 new)
Hawai’i County: 1091
Honolulu County: 12617
Kaua’i County: 60
Maui County: 417†
Pending: 0
Residents diagnosed outside of Hawai‘i: 48
Required Hospitalization: 1033‡
Hawaii deaths: 203
Released from Isolation: 11188§
Cumulative totals as of 12:00pm, October 21, 2020


In [5]:
vals = []
vals.append(parse(soup.find(string=re.compile('Cumulative totals as of'))[23:]).strftime('%m/%d/%Y'))
for t in soup('dd'):
    for d in t(class_='value'):
        vals.append(re.findall(r'\b\d+\b',t.get_text())[0])
vals

['10/21/2020',
 '14233',
 '1091',
 '12617',
 '60',
 '417',
 '0',
 '48',
 '1033',
 '203',
 '11188']

In [6]:
print(vals[0]+','+vals[1]+','+vals[3]+','+vals[2]+','+vals[5]+','+vals[4]+','+vals[7])

10/21/2020,14233,12617,1091,417,60,48
